In [1]:
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np
# Bounding box
yMin = 52.359
yMax = 52.854
xMin = 13.189
xMax = 13.625

step = 0.001
padding = [6,6,8,9]

xMin = xMin-padding[0]*step
xMax = xMax+padding[1]*step
yMin = yMin-padding[2]*step
yMax = yMax+padding[3]*step

print(f"xMin {xMin} - xMax {xMax} - yMin {yMin} - yMax {yMax}")

windowsize = 21


cols = np.arange(xMin,xMax+step,step) # Lon
rows = np.arange(yMin,yMax+step,step)[::-1] # Lat

pool_lons = cols[::windowsize]
pool_lats = rows[::windowsize]

polygons = []
lats = []
lons = []
for i in range(len(pool_lons)):
    for j in range(len(pool_lats)):
        x_left = pool_lons[i]
        y_up = pool_lats[j]
        x_right = cols[-1] if i==pool_lons.shape[0]-1 else pool_lons[i+1]
        y_bot = rows[-1] if j==pool_lats.shape[0]-1 else pool_lats[j+1]
        polygons.append(Polygon([(x_left,y_up), (x_right, y_up), (x_right,y_bot),(x_left, y_bot)]))
        lats.append(j)
        lons.append(i)
grid = gpd.GeoDataFrame({
    'geometry':polygons,
    'x': lons,
    'y': lats,
},crs="EPSG:4326")

xMin 13.183 - xMax 13.631 - yMin 52.351 - yMax 52.863


In [16]:
data = grid.to_json(show_bbox=True, na="drop")
with open("grid.geo.json","w") as f:
    f.writelines(data)

In [18]:
x = 13
pool_cols[x:x+2]  # x

array([13.456, 13.477])

In [19]:
y = 5
pool_rows[y:y+2]  # y

array([52.758, 52.737])

In [17]:
grid[(grid['x']==13) & (grid['y']==5)].geometry

330    POLYGON ((13.45600 52.75800, 13.47700 52.75800...
Name: geometry, dtype: geometry

In [8]:
# Save a part of polygons
import os
FILE_PATH = r"C:\Users\jingyli\OwnDrive\IPA\web_json"

x_center = 12
y_center = 12

# x_list = np.arange(x_center-1,x_center+2)
# y_list = np.arange(y_center-1,y_center+2)
x_list = [x_center]
y_list = [y_center]
single_polygons = []
xs = []
ys = []
for i in x_list:
    for j in y_list:
        x_left = pool_lons[i]
        y_up = pool_lats[j]
        x_right = cols[-1] if i==pool_lons.shape[0]-1 else pool_lons[i+1]
        y_bot = rows[-1] if j==pool_lats.shape[0]-1 else pool_lats[j+1]

        single_polygons.append(Polygon([(x_left,y_up), (x_right, y_up), (x_right,y_bot),(x_left, y_bot)]))
        xs.append(i)
        ys.append(j)
        
grid = gpd.GeoDataFrame({
    'geometry':single_polygons,
    'x': xs,
    'y': ys,
},crs="EPSG:4326")
data = grid.to_json(show_bbox=True, na="drop")
with open(os.path.join(FILE_PATH,f"{y_center}-{x_center}.geo.json"),"w") as f:
    f.writelines(data)